## Disclaimer & Copyright

Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Gemini - Image analysis

Feedback : shins777@gmail.com. 

* This notebook explain about how to use Gemini to understand image in multimodality features of Gemini.  
* Multimodality is one of the core capabilities in the latest foundation models. 
* Gemini models are fundamentally built on this multimodality. 

### 라이브러리 설치

In [2]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [3]:
from IPython.display import display, Markdown

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [4]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


### Set the environment on GCP Project 
* Configure project information 
* Recommend to use the latest model. 

In [5]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.

In [6]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

### Multimodality basic code

In [10]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

def get_obj_content(coding='utf-8', obj_loc=""):
    with open(obj_loc, 'rb') as f:
        obj_data = base64.b64encode(f.read()).decode(coding)
        return obj_data

def generate(file_path:str, query:str, mime_type:str):

    pdf_obj = get_obj_content(obj_loc=file_path)
    doc_obj = Part.from_data(data=base64.b64decode(pdf_obj), mime_type=mime_type)
  
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }
  
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [doc_obj, query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

In [12]:
file_path = "/home/admin_/google_gen_ai_sample/contents/images/Kospi.png"
mime_type = "image/png"

prompt = """이미지에 대해서 아래 내용을 중심으로 자세하게 설명해주세요.
1. 몇일날짜의 정보입니다.
2. 52주 최고가는 얼마인가요 ?
3. 그래프가 생성된 다음날의 상황은 어떻게 예측하나요 ? 이유도 알려주세요.
4. 이 그래프는 일반기업의 그래프인가요 ?
5. 이 그래프는 어떤 데이터를 기반으로 만들어졌나요 ?
"""

outcome = generate(file_path, prompt, mime_type)

print(outcome)



1. 이 그래프는 3월 7일의 정보입니다.
2. 52주 최고가는 2,694.80 입니다.
3. 이 그래프는 KOSPI 지수를 나타내는 것으로, 한국 주식 시장의 전반적인 흐름을 보여줍니다. 최근 흐름을 보면 등락을 반복하고 있지만, 전반적으로 상승 추세를 보이고 있습니다. 따라서 다음 날에도 상승할 가능성이 높습니다. 다만, 시장 상황에 따라 하락할 수도 있다는 점은 유의해야 합니다.
4. 이 그래프는 일반기업의 그래프가 아닙니다. 한국 주식 시장의 대표 지수인 KOSPI 지수를 나타내는 그래프입니다.
5. 이 그래프는 한국거래소(KRX)에서 제공하는 KOSPI 지수의 실시간 데이터를 기반으로 만들어졌습니다.
